# Modelabilidade dos dados

Modelos de QSAR trabalham com o princípio da similaridade: compostos similares devem apresentar propriedades similares. Dessa forma, considere um conjunto de dados contendo estruturas químicas e uma propriedade a ser modelada (por exemplo, toxicidade, potência, etc). Calcular a **modelabilidade** desses dados significa identificar se, em geral, compostos parecidos possuem propriedades similares. 

Idealmente, casos em que compostos similares apresentam propriedades distintas são raros, e recebem o nome de *activity cliffs* (veja o Notebook [Encontrando activity cliffs usando fingerprints](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Encontrando%20activity%20cliffs%20usando%20fingerprints.ipynb)). Apesar de fornecerem informações importantes a respeito da propriedade em questão, os *activity cliffs* prejudicam muito a preditividade de modelos de aprendizado de máquina, e não é possível esperar bons modelos preditivos de conjuntos de dados ricos em *cliffs*.

## Índice de modelabilidade (MODI)

No artigo [Data Set Modelability by QSAR](https://pubs.acs.org/doi/10.1021/ci400572x) foi proposta uma medida da modelabilidade de um conjunto de dados para QSAR: um valor denominado de índice de modelabilidade (*MODelability Index* = **MODI**).

O valor de MODI é calculado para um conjunto de dados contendo $N$ classes usando uma razão ponderada por classe do número de pares de compostos vizinhos mais próximos com a mesma classe de atividade versus o número total de pares:

$$ MODI = {1 \over N} * {\sum_{i=1}^{K} {N_i^{same} \over N_i^{total}}} $$

Vamos separar o método em etapas para deixar mais claro:

- Para cada classe $i$:
	- Inicie uma variável $N_i^{same}$ com o valor 0
	- Denominamos como $N_i^{total}$ o número de compostos pertencentes à classe $i$. Por exemplo, em um dataset binário com 100 compostos na classe 0 e 200 compostos na classe 1, $N_0^{total}$ = 100 e $N_1^{total}$ = 200
	- Para cada composto da classe, identifique o vizinho mais próximo (no *dataset* completo). Os autores demonstraram que tanto descritores físico-químicos como *fingerprints* podem ser usados para identificar os vizinhos.
	- Se os valores forem diferentes, temos um *activity cliff*
	- Se os valores forem iguais, adicione 1 ao valor de $N_i^{same}$
- O valor final de $N_i^{same}/N_i^{total}$ é o **MODI para a classe i**
- Some os valores de $N_i^{same}/N_i^{total}$ de todas as classes e divida pelo número de classes - esse é o **MODI do conjunto de dados completo**, mostrado na equação acima. 

Portanto, um conjunto de dados binário terá três valores de MODI: um para cada classe (0 e 1) e mais um para o conjunto completo (este será a média dos dois valores das classes). Da mesma forma, um conjunto de dados multiclasse terá um valor de MODI para cada classe, mais um total.

Um ponto importante a ser considerado é que o vizinho mais próximo de um composto não necessariamente representará uma estrutura análoga. Isso é ainda mais relevante quando tentamos modelar uma coleção de compostos que foi montada pensando em uma vasta exploração do espaço químico, e que por isso apresentará compostos com pouca similaridade entre si. O que os autores observaram é que, como regra geral, conjuntos de dados não modeláveis (com valores MODI relativamente baixos) também apresentam valores médios de índice de Tanimoto (Tc) relativamente baixos para todos os pares de vizinhos mais próximos e, vice-versa, conjuntos de dados modeláveis apresentam valores médios de Tc relativamente altos. Isso é explicado da seguinte forma: conjuntos de dados mais diversos apresentam vizinhos mais dissimilares, e isso faz com que a probabilidade de que as moléculas pertençam à mesma classe seja menor do que se os vizinhos fossem mais parecidos. Dessa forma, a dissimilaridade entre as moléculas, um fator que também prejudica a modelabilidade dos dados, acaba sendo considerada no cálculo do valor de MODI.

## MODI - um módulo em Python para cálculo do índice de modelabilidade

Os autores do artigo disponibilizaram na sua página do GitHub um módulo para cálculo do valor de MODI. O programa também se chama [MODI](https://github.com/molecularmodelinglab/MODI).

No momento, o valor de MODI pode ser calculado apenas para problemas de classificação (binários ou multiclasse). Há também uma função para cálculo de *fingerprints* e identificação dos vizinhos pela distância Euclidiana (usando o método [KDTree](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html)). Como já discuti em [outro Notebook](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Clustering%20(Agrupamento).ipynb), a distância Euclidiana não é a escolha mais rigorosa para *fingerprints*. De qualquer forma, os resultados do artigo mostraram-se robustos usando essa combinação, e vamos utilizá-la na demonstração abaixo, na qual exploraremos a modelabilidade do *dataset* BBBP do [MoleculeNet](https://moleculenet.org/datasets-1) que contém valores binários referentes à penetração na barreira hematoencefálica (permeabilidade).

## Importar módulos e preprocessar o conjunto de dados



In [1]:
# Importar o dataset BBBP
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.DataStructs import ConvertToNumpyArray

df = pd.read_csv('../datasets/BBBP.csv')
df.head()

,num,name,p_np,smiles
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...


Vamos padronizar as estruturas usando a função apresentada no Notebook [Padronizando estruturas químicas](https://github.com/rflameiro/Python_e_Quiminformatica/blob/main/Quiminformatica/Padronizando%20estruturas%20qu%C3%ADmicas.ipynb):

In [2]:
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize
from chembl_structure_pipeline import standardizer

def new_standardize_mol(molecule, return_mol=True):
    """
    Input = Objeto RDKitMol
    return_mol: indica se a função vai retornar RDKit Mol (True) ou SMILES (False)
    """
    try:
        # aplicar standardize_mol
        std1_mol = standardizer.standardize_mol(molecule)
        # selecionar o maior fragmento
        desalter = rdMolStandardize.LargestFragmentChooser()
        desalt_mol = desalter.choose(std1_mol)
        # aplicar standardize_mol ao maior fragmento
        std2_mol = standardizer.standardize_mol(desalt_mol)
        # selecionar tautômero com maior score
        te = rdMolStandardize.TautomerEnumerator()
        curated_mol = te.Canonicalize(std2_mol) 
        assert curated_mol != None
        
        if return_mol:
            return curated_mol
        else:
            return Chem.MolToSmiles(curated_mol)
    
    except Exception as e: 
        print(e)
        print("Erro, falha na padronização")
        return


[23:41:18] Initializing Normalizer


In [3]:
len(df)

2050

In [4]:
%%capture
df["rdkit_mol"] = df["smiles"].apply(Chem.MolFromSmiles)

In [5]:
df["rdkit_mol"].isna().sum()

11

Os 11 compostos abaixo não puderam ser convertidos a RDKit Mol e serão descartados

In [6]:
df[df["rdkit_mol"].isna()]

,num,name,p_np,smiles,rdkit_mol
59,60,15,1,O=N([O-])C1=C(CN=C1NCCSCc2ncccc2)Cc3ccccc3,None
61,62,22767,1,c1(nc(NC(N)=[NH2])sc1)CSCCNC(=[NH]C#N)NC,None
391,393,ICI17148,1,Cc1nc(sc1)\[NH]=C(\N)N,None
614,616,5-6,1,s1cc(CSCCN\C(NC)=[NH]\C#N)nc1\[NH]=C(\N)N,None
642,644,12,0,c1c(c(ncc1)CSCCN\C(=[NH]\C#N)NCC)Br,None
645,647,16,1,n1c(csc1\[NH]=C(\N)N)c1ccccc1,None
646,648,17,0,n1c(csc1\[NH]=C(\N)N)c1cccc(c1)N,None
647,649,18,0,n1c(csc1\[NH]=C(\N)N)c1cccc(c1)NC(C)=O,None
648,650,19,0,n1c(csc1\[NH]=C(\N)N)c1cccc(c1)N\C(NC)=[NH]\C#N,None
649,651,2,1,s1cc(nc1\[NH]=C(\N)N)C,None


In [7]:
# Selecionando compostos que completaram a conversão a RDKit Mol
df = df[~df["rdkit_mol"].isna()]

In [8]:
%%capture
# Calculando os SMILES padronizados
df["std_smiles"] = df["rdkit_mol"].apply(lambda mol: new_standardize_mol(mol, return_mol=False))

In [9]:
# Antes da padronização
df["smiles"].duplicated(keep=False).sum()

0

In [10]:
# Depois da padronização
df["std_smiles"].duplicated(keep=False).sum()

105

Com a padronização, 105 entradas passam a ser duplicatas. Essas entradas podem corresponder a sais com contraíons diferentes. Como isso pode influenciar o valor de BBBP, vamos remover todas as entradas duplicadas.

In [11]:
df = df[~df["std_smiles"].duplicated(keep=False)]
df.shape

(1934, 6)

In [12]:
# Salvar o dataset para uso futuro
df.to_csv('../datasets/BBBP_curated.csv')

Finalmente, convertemos os SMILES padronizados a RDKit Mol para cálculo dos *fingerprints*:

In [13]:
df["rdkit_mol"] = df["std_smiles"].apply(Chem.MolFromSmiles)

## Usando o módulo MODI

Para usar o MODI é preciso ter o RDKit instalado. Em seguida, baixe o arquivo `modi.py` e salve-o, preferencialmente, na mesma pasta em que o seu Notebook se encontra. Link para baixar: https://github.com/molecularmodelinglab/MODI/blob/main/modi.py

Em seguida, usamos o comando a seguir para indicar ao Python que ele pode importar módulos da pasta local:

In [14]:
import sys 

sys.path.append(".")  # adiciona a pasta local (.) ao path do Python

# Também é possível adicionar uma pasta usando seu path absoluto:
# import os
# sys.path.append(os.path.abspath("/Users/username/Folder1/Folder2"))

Agora é só importar:

In [15]:
import modi
from modi import get_morgan_finger

O módulo `modi` possui apenas três funções para cálculo do valor de MODI. As três calculam o mesmo valor, e variam apenas no formato do *input*:
- modi()
- modi_from_sdf()
- modi_from_csv()

Como os nomes indicam, as duas últimas funções permitem passar os arquivos no formato .sdf ou .csv em vez de um *dataset* pré-carregado. Elas podem ser úteis quando se usa o `modi.py` a partir da linha de comando. Caso opte por usá-lo dessa forma mais avançada, você pode chamar `modi.py -h` para ver a lista de parâmetros e opções.

Vamos mostrar a utilização da primeira função:

## modi()

Use essa função quando estiver trabalhando com seu conjunto de dados no Jupyter Notebook. É necessário criar dois *numpy arrays*: 
- um *array* no qual as linhas correspondem a compostos químicos, e as colunas, aos descritores (no caso, vamos usar *Morgan fingerprints*, já que o próprio MODI disponibiliza a função `get_morgan_finger`) 
- um *array* contendo os valores das propriedades

In [16]:
# Gerar fingerprints usando a função fornecida no modi.py
fp_array = np.vstack(df["rdkit_mol"].apply(get_morgan_finger).to_numpy())

In [17]:
# Gerar array com os valores das propriedades
prop_array = np.array(df["p_np"])

In [18]:
# Calculando o valor de MODI
modi_value = modi.modi(fp_array, prop_array)

In [19]:
modi_value

0.7853531319155986

Como interpretar esse valor? Os autores calcularam os índices de modelabilidade para mais de 100 conjuntos de dados, e o valor de **0,65** foi identificado como um valor de corte razoável para separar os conjuntos de dados não modeláveis dos modeláveis. 

Não leve isso ao pé da letra, mas tenha em mente que se o MODI do seu conjunto de dados for menor que 0,65, há uma boa chance de que seu conjunto de dados será mais difícil de modelar devido à presença de muitos *activity cliffs*.

A função `modi()` também possui o parâmetro opcional `return_class_contribution`, que os autores sugerem alterar para "True" no caso de *datasets* desbalanceados. Vejamos se é o caso:

In [20]:
print(f"Classe 0: {len(prop_array) - prop_array.sum()}")
print(f"Classe 1: {prop_array.sum()}")

Classe 0: 458
Classe 1: 1476


Nosso *dataset* apresenta um desbalanceamento de aproximadamente 1:3. Vamos ver como o valor de MODI se altera ao usarmos `return_class_contribution=True`. Note que a função agora retorna uma tupla, e podemos guardar os resultados em duas variáveis.

In [21]:
modi_value, class_contrib = modi.modi(fp_array, prop_array, return_class_contribution=True)

In [22]:
modi_value

0.7853531319155986

In [23]:
class_contrib

[(0, 0.631004366812227), (1, 0.9397018970189702)]

Veja que o MODI da classe 0 é menor que da classe 1. Isso significa que cerca de 37% (100% - 63%) dos vizinhos mais próximos dos compostos da classe 0 pertencem à classe 1, mas apenas 6% dos compostos da classe 1 apresentam vizinhos da classe 0. Isso nos sugere que o modelo terá mais dificuldade em modelar a classe 0, ou seja, podemos esperar um número maior de falsos negativos do que de falsos positivos. Vejamos se isso de fato ocorre: 

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

X, y = fp_array, prop_array

# Vamos dividir os dados em conjuntos de treinamento e teste mantendo as classes equilibradas
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_index, test_index = next(sss.split(X, y))

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Ajustando um modelo de regressão logística
model = LogisticRegression()
model.fit(X_train, y_train)

# Calculando as predições para o conjunto de teste
y_pred = model.predict(X_test)

# Step 6: Generate a confusion matrix and label the matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

print("Confusion Matrix:")
print("True Positives:", tp)
print("False Positives:", fp)
print("True Negatives:", tn)
print("False Negatives:", fn)

Confusion Matrix:
True Positives: 279
False Positives: 32
True Negatives: 60
False Negatives: 16


In [25]:
print(f"Porcentagem de falsos negativos sobre os preditos negativos (classe 0): {fn*100/(fn+tn):.2f}")
print(f"Porcentagem de falsos positivos sobre os preditos positivos (classe 1): {fp*100/(fp+tp):.2f}")

Porcentagem de falsos negativos sobre os preditos negativos (classe 0): 21.05
Porcentagem de falsos positivos sobre os preditos positivos (classe 1): 10.29


Confirmamos que há mais falsos negativos do que falsos positivos, em proporção aos valores preditos. Note que o resultado também reflete o desbalanceamento dos dados. Dessa forma, concluímos que pode ser interessante aumentar o número de compostos da classe 0 para obtermos um conjunto mais modelável e modelos mais preditivos.